In [161]:
import numpy as np
import lda
import lda.datasets

X = lda.datasets.load_reuters()
matrix = X
vocabulary = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()
X.shape

(395, 4258)

In [133]:
# Dirichlet priors
alpha      = 1
beta       = 1


#set the numbers
n_docs = X.shape[0]
n_topics = 20
vocab_size = len(vocabulary)

In [134]:
def word_indices(vec):
    for idx in vec.nonzero()[0]:
        for i in range(int(vec[idx])):
            yield idx

In [135]:
#Initialize
# number of times document m and topic z co-occur
nmz = np.zeros((n_docs, n_topics))
# number of times topic z and word w co-occur
nzw = np.zeros((n_topics, vocab_size))
nm = np.zeros(n_docs)
nz = np.zeros(n_topics)
topics = {}
for m in range(n_docs):
    # i is a number between 0 and doc_length-1
    # w is a number between 0 and vocab_size-1
    for i, w in enumerate(word_indices(matrix[m, :])):
        # choose an arbitrary topic as first topic for word i
        z = np.random.randint(n_topics)
        nmz[m,z] += 1
        nm[m] += 1
        nzw[z,w] += 1
        nz[z] += 1
        topics[(m,i)] = z

In [15]:
doctopic = np.zeros(n_docs);
topic_ratio = []
for m in range(n_docs):
    topicss = []
    for i, w in enumerate(word_indices(matrix[m, :])):
        topicss.append(topics[(m,i)])
    doctopic[m] = max(set(topicss), key=topicss.count)
    topic_ratio.append(topicss.count(doctopic[m]) / len(topicss))

In [160]:
topic = 14
idx = np.where(doctopic == topic)
#titles[idx]
idx = idx[0]
a = list(titles)
k = [a[i] for i in idx]
print('Topic', topic, 'Titles')
k[:15]

Topic 14 Titles


["31 BRAZIL: Nobel Prize winner sides with Brazil's landless. BRASILIA 1996-09-10",
 '61 CANADA: Former Quebec premier fought for Canadian unity. QUEBEC CITY 1996-10-02',
 '64 GERMANY: Vargas Llosa urges sanctions on rights abuses. FRANKFURT 1996-10-06',
 '79 INDONESIA: Nobel peace awards put East Timor in spotlight. JAKARTA 1996-10-11',
 '80 NORWAY: Nobel peace award wins praise outside indonesia. OSLO 1996-10-11',
 "81 VATICAN: Bishop's Nobel prize sweet satisfaction for Pope. VATICAN CITY 1996-10-11",
 '82 NORWAY: Winners of Nobel Peace Prize since 1970. OSLO 1996-10-11',
 '86 INDONESIA: Nobel laureate invited to gathering for Suharto. JAKARTA 1996-10-14',
 '126 INDONESIA: Norway issues Timorese Nobel laureate a visa. JAKARTA 1996-11-17',
 '145 ITALY: Nobel laureate Belo says peace will be his message. ROME 1996-12-06',
 '151 NORWAY: East Timor activist shares peace prize. OSLO 1996-12-10',
 "196 LIBERIA: Liberia's Taylor remarries, offers toast for peace. GBANGA, Liberia 1997-01-28

In [107]:
#This is to display most frequent words in topics
topic_word_count = np.zeros((n_topics,vocab_size))
for m in range(n_docs):
    for i, w in enumerate(word_indices(matrix[m, :])):
        topic_word_count[topics[(m,i)], w] = topic_word_count[topics[(m,i)], w] + 1

In [145]:
for i in range(n_topics):
    wordsintopic = topic_word_count[i,:]
    mostindices = (-wordsintopic).argsort()[:8]
    voclist = list(vocabulary)
    mostvocs = [voclist[k] for k in mostindices.tolist()]
    print('Topic',i,'=',mostvocs)

Topic 0 = ['elvis', 'music', 'first', 'fans', 'concert', 'film', 'years', 'tour']
Topic 1 = ['charles', 'prince', 'diana', 'king', 'royal', 'queen', 'bowles', 'parker']
Topic 2 = ['television', 'told', 'says', 'show', 'people', 'group', 'year', "n't"]
Topic 3 = ['buddhist', 'vietnam', 'law', 'church', 'dent', 'friday', 'temple', 'told']
Topic 4 = ['bernardin', 'cardinal', 'church', 'catholic', 'cancer', 'death', 'life', 'chicago']
Topic 5 = ['mother', 'teresa', 'order', 'heart', 'charity', 'nuns', 'calcutta', 'sister']
Topic 6 = ['church', 'wright', 'million', 'kennedy', 'sale', 'former', 'bishop', 'letters']
Topic 7 = ['germany', 'film', 'german', 'nazi', 'letter', 'people', 'scientology', 'against']
Topic 8 = ['city', 'west', 'salonika', 'cultural', 'irish', 'israel', 'byzantine', 'israeli']
Topic 9 = ['pocahontas', 'bun', 'yet', 'cuba', 'done', 'refused', 'course', 'black']
Topic 10 = ['harriman', 'u.s', 'clinton', 'churchill', 'president', 'paris', 'ambassador', 'france']
Topic 11 

In [136]:
#Gibbs steps
maxiter = 2000
for it in range(maxiter):
    if it%10==0:
        print(it)
    for m in range(n_docs):
        for i, w in enumerate(word_indices(matrix[m, :])):
            z = topics[(m,i)]
            nmz[m,z] -= 1
            nm[m] -= 1
            nzw[z,w] -= 1
            nz[z] -= 1
            left = (nzw[:,w] + beta) / (nz + beta * vocab_size)
            right = (nmz[m,:] + alpha) / (nm[m] + alpha * n_topics)
            p_z = left * right
            # normalize to obtain probabilities
            p_z /= np.sum(p_z)
            z = np.random.multinomial(1,p_z).argmax()

            nmz[m,z] += 1
            nm[m] += 1
            nzw[z,w] += 1
            nz[z] += 1
            topics[(m,i)] = z

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
